## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatheryPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
0,0,Sisimiut,GL,66.9395,-53.6735,26.73,87,100,5.35,light snow
1,1,Ushuaia,AR,-54.8000,-68.3000,37.06,75,20,18.41,few clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,39.31,70,75,10.36,broken clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,68.63,60,74,7.16,broken clouds
4,4,Nome,US,64.5011,-165.4064,45.16,61,0,4.61,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest temperature you want for a vacation?"))
max_temp = float(input("What is the highest temperature you want for a vacation?"))

What is the lowest temperature you want for a vacation?75
What is the highest temperature you want for a vacation?82


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacay_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
vacay_cities.count()

City_ID               115
City                  115
Country               115
Lat                   115
Lng                   115
Max Temp              115
Humidity              115
Cloudiness            115
Wind Speed            115
Current Conditions    115
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

Vacay_cities= vacay_cities.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Vacay_cities[["City", "Country", "Max Temp", "Current Conditions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
6,Atuona,PF,78.10,scattered clouds,-9.8000,-139.0333,
11,Rikitea,PF,75.76,light rain,-23.1203,-134.9692,
13,Ndele,CF,79.88,overcast clouds,8.4109,20.6473,
22,Butaritari,KI,81.50,scattered clouds,3.0707,172.7902,
36,Arraial Do Cabo,BR,76.96,few clouds,-22.9661,-42.0278,
40,Faya,SA,75.72,clear sky,18.3851,42.4509,
42,Kavieng,PG,80.98,overcast clouds,-2.5744,150.7967,
43,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
48,Kloulklubed,PW,77.50,light rain,7.0419,134.2556,
59,Papar,MY,80.60,overcast clouds,5.7333,115.9333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    params["location"] = f"{lat}, {lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping")

hotel_df.head(10)
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
6,Atuona,PF,78.10,scattered clouds,-9.8000,-139.0333,Villa Enata
11,Rikitea,PF,75.76,light rain,-23.1203,-134.9692,People ThankYou
13,Ndele,CF,79.88,overcast clouds,8.4109,20.6473,
22,Butaritari,KI,81.50,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
36,Arraial Do Cabo,BR,76.96,few clouds,-22.9661,-42.0278,Pousada Porto Praia
40,Faya,SA,75.72,clear sky,18.3851,42.4509,املاك خالد الشعشوع
42,Kavieng,PG,80.98,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
43,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,Casa Tia Villa
48,Kloulklubed,PW,77.50,light rain,7.0419,134.2556,Storyboard Beach Resort
59,Papar,MY,80.60,overcast clouds,5.7333,115.9333,Koposizon Homestay


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df['Hotel Name']=hotel_df['Hotel Name'].str.strip().replace('',np.nan)

clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)

export_file = "weather_data/WeatherPy_Vacations.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(export_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Conditions}</dd>
<dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
#locations = clean_hotel_info[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))